# Best Model Neural Network

In [1]:
import sys
sys.path.append("../src")

In [2]:
from features.features_dataset import FeaturesDataset
from models.fully_connected_classifier import FullyConnectedClassifier
from collections import Counter
from torch.utils.data import DataLoader

In [3]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Import Dataset

In [4]:
norm_letters_signal_dataset_path= r"..\data\processed\feat_norm_letters_1_1_dataset_mean.pkl"
feat_dataset = FeaturesDataset.load(file_path= norm_letters_signal_dataset_path)
feat_dataset.features.shape

(8390, 187)

## Prepare Data

In [5]:
feat_dataset.prepare_dataset(drop_constant= False, drop_flatness= True)

## RQ1

Is there any difference in the signals when someone is performing eurythmy?

In [6]:
feat_dataset.features.head()

,id_measurement,id_performance,datetime,plant,generation,num_eurythmy,eurythmy_letter,lib_mfcc_1_avg,lib_mfcc_2_avg,lib_mfcc_3_avg,lib_mfcc_4_avg,lib_mfcc_5_avg,lib_mfcc_6_avg,lib_mfcc_7_avg,lib_mfcc_8_avg,lib_mfcc_9_avg,lib_mfcc_10_avg,lib_mfcc_11_avg,lib_mfcc_12_avg,lib_mfcc_13_avg,lib_mfcc_1_std,lib_mfcc_2_std,lib_mfcc_3_std,lib_mfcc_4_std,lib_mfcc_5_std,lib_mfcc_6_std,lib_mfcc_7_std,lib_mfcc_8_std,lib_mfcc_9_std,lib_mfcc_10_std,lib_mfcc_11_std,lib_mfcc_12_std,lib_mfcc_13_std,zcr_mean,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,spectral_entropy_mean,spectral_flux_mean,spectral_rolloff_mean,mfcc_1_mean,mfcc_2_mean,mfcc_3_mean,mfcc_4_mean,mfcc_5_mean,mfcc_6_mean,mfcc_7_mean,mfcc_8_mean,mfcc_9_mean,mfcc_10_mean,mfcc_11_mean,mfcc_12_mean,mfcc_13_mean,chroma_1_mean,chroma_2_mean,chroma_3_mean,chroma_4_mean,chroma_5_mean,chroma_6_mean,chroma_7_mean,chroma_8_mean,chroma_9_mean,chroma_10_mean,chroma_11_mean,chroma_12_mean,chroma_std_mean,delta zcr_mean,delta energy_mean,delta energy_entropy_mean,delta spectral_centroid_mean,delta spectral_spread_mean,delta spectral_entropy_mean,delta spectral_flux_mean,delta spectral_rolloff_mean,delta mfcc_1_mean,delta mfcc_2_mean,delta mfcc_3_mean,delta mfcc_4_mean,delta mfcc_5_mean,delta mfcc_6_mean,delta mfcc_7_mean,delta mfcc_8_mean,delta mfcc_9_mean,delta mfcc_10_mean,delta mfcc_11_mean,delta mfcc_12_mean,delta mfcc_13_mean,delta chroma_1_mean,delta chroma_2_mean,delta chroma_3_mean,delta chroma_4_mean,delta chroma_5_mean,delta chroma_6_mean,delta chroma_7_mean,delta chroma_8_mean,delta chroma_9_mean,delta chroma_10_mean,delta chroma_11_mean,delta chroma_12_mean,delta chroma_std_mean,zcr_std,energy_std,energy_entropy_std,spectral_centroid_std,spectral_spread_std,spectral_entropy_std,spectral_flux_std,spectral_rolloff_std,mfcc_1_std,mfcc_2_std,mfcc_3_std,mfcc_4_std,mfcc_5_std,mfcc_6_std,mfcc_7_std,mfcc_8_std,mfcc_9_std,mfcc_10_std,mfcc_11_std,mfcc_12_std,mfcc_13_std,chroma_1_std,chroma_2_std,chroma_3_std,chroma_4_std,chroma_5_std,chroma_6_std,chroma_7_std,chroma_8_std,chroma_9_std,chroma_10_std,chroma_11_std,chroma_12_std,chroma_std_std,delta zcr_std,delta energy_std,delta energy_entropy_std,delta spectral_centroid_std,delta spectral_spread_std,delta spectral_entropy_std,delta spectral_flux_std,delta spectral_rolloff_std,delta mfcc_1_std,delta mfcc_2_std,delta mfcc_3_std,delta mfcc_4_std,delta mfcc_5_std,delta mfcc_6_std,delta mfcc_7_std,delta mfcc_8_std,delta mfcc_9_std,delta mfcc_10_std,delta mfcc_11_std,delta mfcc_12_std,delta mfcc_13_std,delta chroma_1_std,delta chroma_2_std,delta chroma_3_std,delta chroma_4_std,delta chroma_5_std,delta chroma_6_std,delta chroma_7_std,delta chroma_8_std,delta chroma_9_std,delta chroma_10_std,delta chroma_11_std,delta chroma_12_std,delta chroma_std_std,zero_crossing_rate,root_mean_square_energy,slope_sign_changes_ratio,hjorth_mobility,hjorth_complexity,mean,variance,standard_deviation,interquartile_range,skewness,kurtosis,dfa
0,1,1,2023-04-29,salad,1,1,A1,-85.835655,90.291260,27.150677,24.450397,17.381523,14.476572,11.628652,10.253779,9.009523,8.263220,7.481691,6.924080,6.325689,87.071404,0.980359,0.953386,0.947235,0.955398,0.965027,0.979433,0.972912,0.969672,0.946374,0.945209,0.952646,0.956299,0.000111,0.175157,3.270264,0.034481,0.083165,0.013287,0.040835,0.001133,-60.331463,1.260832,0.148831,0.233323,0.100410,0.115823,0.063385,0.059853,0.024441,0.031632,0.025881,0.039124,0.033931,0.000136,0.000022,0.003935,0.000123,0.192401,0.000078,0.012127,0.000014,0.000043,0.000117,0.000694,0.000034,0.055544,0.0,-0.000614,-0.000181,0.000369,0.000981,2.965442e-05,1.837984e-04,0.000000e+00,0.388925,0.012609,0.001508,0.002126,0.001266,0.001514,0.000789,0.001239,0.001191,0.001494,0.001671,0.001324,0.000511,1.755055e-07,2.739468e-08,4.760306e-06,2.778529e-08,-7.296253e-06,8.169879e-08,6.575731e-06,1.506539e-08,1.250590e-07,3.546288e-07,2.423069e-06,5.853458e-08,-2.376859e-06,0.000230,0.097363,0.074493,0.028377,0.066244,0.026147,0.058638,0.002

In [7]:
rq1= feat_dataset.return_subset_given_research_question(1)

In [8]:
train_feat_dataset, _, test_feat_dataset= rq1.split_dataset(split_by_wav= False,test_size= 0.2, val_size= 0, random_state= True)

### Data preparation

In [9]:
# Reduce the features that are correlated in the training data
train_cols, feat_stats= train_feat_dataset.reduce_features_based_on_target(corr_threshold=0.8)
test_feat_dataset.keep_only_specified_variable_columns(train_cols)

# Normalize features ?????
normalization_params= train_feat_dataset.normalize_features()
test_feat_dataset.apply_normalization(normalization_params)

Reduced variable features from initial count to 43.
Variable features were properly normalized using 'zscore' method.
Applied z-score normalization.


C:\Users\alvar\Documents\GitHub\Plant-Reactivity-Analysis\notebooks\../src\features\features_dataset.py:418: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  feature_stats = feature_stats.applymap(lambda x: f"{x:.2e}")


In [10]:
feat_stats.head(150)

,0,1,p_value
kurtosis,1.86e+01,1.83e+00,nan
skewness,-8.00e-02,8.99e-02,nan
lib_mfcc_12_avg,6.44e+00,6.76e+00,9.86e-45
spectral_spread_mean,1.21e-01,9.96e-02,9.80e-119
lib_mfcc_3_avg,2.47e+01,2.59e+01,9.76e-45
zcr_mean,1.85e-04,2.63e-04,9.60e-02
delta spectral_spread_std,6.44e-02,7.49e-02,9.50e-61
delta zcr_std,5.93e-04,5.96e-04,9.50e-01
lib_mfcc_10_avg,7.51e+00,8.03e+00,9.38e-98
mfcc_11_std,8.72e-02,8.72e-02,9.37e-01


In [11]:
train_feat_dataset.objective_features.head()

,lib_mfcc_1_avg,lib_mfcc_2_avg,lib_mfcc_3_avg,lib_mfcc_13_avg,lib_mfcc_1_std,lib_mfcc_3_std,lib_mfcc_13_std,energy_mean,energy_entropy_mean,mfcc_9_mean,mfcc_11_mean,mfcc_13_mean,delta zcr_mean,delta energy_mean,delta energy_entropy_mean,delta spectral_centroid_mean,delta spectral_flux_mean,delta mfcc_3_mean,delta mfcc_5_mean,delta mfcc_6_mean,delta mfcc_7_mean,delta mfcc_8_mean,delta mfcc_9_mean,delta mfcc_10_mean,delta mfcc_11_mean,delta mfcc_12_mean,delta mfcc_13_mean,delta chroma_1_mean,delta chroma_10_mean,energy_std,spectral_spread_std,spectral_rolloff_std,mfcc_11_std,delta zcr_std,delta energy_std,delta spectral_centroid_std,root_mean_square_energy,slope_sign_changes_ratio,hjorth_complexity,mean,variance,interquartile_range,dfa
0,1.377359,0.061525,-1.029548,-0.233882,0.421467,0.145063,3.124360,-0.733673,-0.268481,0.652231,1.203061,0.745552,0.004585,-0.466351,0.021075,-0.036732,0.168898,0.100105,-0.545485,-0.939821,-0.737115,0.050218,-0.258694,-0.980795,-0.301172,1.436076,0.692711,0.011268,0.002831,0.033069,-0.574704,0.312955,-0.395965,0.099674,0.310241,0.421151,1.632252,0.493309,-0.722224,1.820063,0.929386,0.790848,0.345436
1,1.753532,0.065833,-0.944990,-0.094143,-1.386619,-0.122261,1.773266,-1.243132,-0.902340,0.819088,0.683036,0.567494,0.748844,-0.003141,-0.784841,1.212088,0.265458,0.219724,0.196796,-0.014682,-0.349859,-0.216845,0.328264,0.194414,-0.237414,-0.123792,0.539527,2.808774,1.866594,-0.941966,-0.797017,1.117926,0.400707,0.477048,0.225650,1.197648,0.380117,0.159412,-0.734176,-0.434348,0.297223,0.211336,-0.402622
2,0.257775,0.496157,0.330683,0.199462,-0.202979,-0.072261,-0.142530,-0.986109,-0.280397,0.891003,0.695340,-0.148989,0.004585,-0.899053,-0.597608,0.543013,-0.074099,0.122335,0.136338,-0.063935,-0.020577,-0.042186,0.170478,0.191330,0.094049,-0.228889,-0.497537,0.130337,0.192393,-0.679968,-0.012362,0.841673,-0.188227,0.173989,-0.400068,0.438965,0.309039,-0.122263,0.310940,-1.468057,-0.333772,-0.228814,0.753765
3,-1.079190,-0.119156,0.495905,1.020374,1.602477,1.271753,0.381257,-0.774608,0.034866,-2.641702,-1.095509,-1.811671,0.004585,0.058783,0.801992,-1.043029,-0.207851,-0.582299,-0.608932,-0.628397,-0.359062,-0.307344,-0.261815,-0.192146,-0.209867,-0.343258,-0.300428,-0.218660,-0.086464,-0.848564,1.024378,0.240558,-0.229942,0.029471,-0.749113,0.616853,-0.842293,-0.424004,-0.214136,-0.588543,-0.495265,-1.091729,-0.581189
4,0.885542,0.119408,-0.895518,-1.633319,-1.519433,-0.137335,-0.463067,-0.131292,-0.492647,0.440384,0.216143,0.742770,-0.807333,0.171728,2.519842,-1.156328,-0.182258,0.147204,0.028075,-0.069313,-0.005800,0.207935,0.196215,0.005832,0.115730,0.387804,0.238197,-1.360750,-0.763255,0.884317,-0.978449,0.622162,-0.933713,0.276103,2.184946,0.844199,-0.129069,0.440095,-0.500644,0.464291,-0.093631,0.508721,0.232533


In [12]:
test_feat_dataset.objective_features.head()

,lib_mfcc_1_avg,lib_mfcc_2_avg,lib_mfcc_3_avg,lib_mfcc_13_avg,lib_mfcc_1_std,lib_mfcc_3_std,lib_mfcc_13_std,energy_mean,energy_entropy_mean,mfcc_9_mean,mfcc_11_mean,mfcc_13_mean,delta zcr_mean,delta energy_mean,delta energy_entropy_mean,delta spectral_centroid_mean,delta spectral_flux_mean,delta mfcc_3_mean,delta mfcc_5_mean,delta mfcc_6_mean,delta mfcc_7_mean,delta mfcc_8_mean,delta mfcc_9_mean,delta mfcc_10_mean,delta mfcc_11_mean,delta mfcc_12_mean,delta mfcc_13_mean,delta chroma_1_mean,delta chroma_10_mean,energy_std,spectral_spread_std,spectral_rolloff_std,mfcc_11_std,delta zcr_std,delta energy_std,delta spectral_centroid_std,root_mean_square_energy,slope_sign_changes_ratio,hjorth_complexity,mean,variance,interquartile_range,dfa
0,1.356651,0.292482,-0.280452,-0.894066,-1.106874,-0.555395,0.277695,-0.845011,-0.253397,0.643787,0.173811,0.966136,0.004585,-0.118803,-0.034703,0.026669,-0.198596,0.012886,0.236682,0.194438,-0.024680,-0.080664,0.013635,-0.116880,-0.124179,0.318651,0.391812,-0.009785,0.014432,-0.143464,-0.487219,-0.014591,0.207162,0.099274,0.154679,0.356471,0.634936,-0.005628,-0.310954,1.033643,0.311260,0.540129,0.111979
1,0.282308,0.489948,0.272371,-0.761095,-0.804739,-0.777955,-0.805219,-0.019760,0.167190,0.776884,1.450099,0.168034,0.004585,-0.163154,0.004365,0.091136,0.116779,0.164831,0.121855,0.184757,0.088217,0.099071,0.188832,0.190922,0.217789,0.224130,0.175478,0.015645,0.003276,-0.115978,-0.713086,-0.566939,0.578745,-0.106953,-0.114060,-0.556193,-0.644136,-0.210611,-0.417199,-0.107156,-0.320649,-0.086657,-0.103384
2,-0.261635,0.598862,0.581525,0.395490,0.007289,-0.688728,-0.314916,-0.366589,0.070332,-0.386942,-0.543903,-1.268221,0.004585,-0.106472,0.017203,0.006301,0.034576,0.018272,0.022319,0.018958,0.016054,0.013055,-0.010004,-0.003202,-0.007450,0.017543,0.015495,0.011505,0.003125,-0.325131,1.224285,0.046430,0.044671,-0.105472,-0.554041,0.016891,-0.458283,-0.383489,-0.185870,0.857124,-0.426266,-0.355593,-0.139369
3,2.211007,0.473628,0.179504,-1.421178,-0.822051,-0.943051,0.366389,-0.940942,-0.870948,-2.058097,-2.122247,-0.996094,0.004585,0.765109,-0.086881,0.370595,-0.203160,0.368842,0.359437,0.457717,0.210595,0.182175,0.282843,0.067277,0.152166,0.170507,0.156176,0.029517,0.019725,-0.943805,2.275016,2.524024,-0.957600,0.115999,-0.074420,2.532580,-0.306755,-0.438361,-0.984233,-0.489501,-0.222505,-1.204628,-2.450852
4,0.060171,0.467523,0.268370,-0.910004,-1.438499,-0.762338,-0.819537,-0.322182,0.061151,0.533057,0.137307,1.379305,0.004585,0.082018,-0.009984,0.265838,-0.091264,0.493620,0.475057,0.477187,0.257992,0.228510,0.191046,0.190319,0.199437,0.238505,0.079801,0.018705,0.005294,-0.275515,0.169786,-0.343472,1.391090,-0.041624,0.127630,-0.179866,-0.576071,-0.272966,-0.071286,0.652729,-0.398084,-0.350360,-0.627211


In [13]:
train_feat_dataset.print_target_distribution()
test_feat_dataset.print_target_distribution()

Counts and Percentages:
Class 0: Count = 3311, Percentage = 49.33%
Class 1: Count = 3401, Percentage = 50.67%
Counts and Percentages:
Class 0: Count = 838, Percentage = 49.94%
Class 1: Count = 840, Percentage = 50.06%


In [14]:
train_loader= DataLoader(train_feat_dataset, batch_size=32, shuffle=True)
test_loader= DataLoader(test_feat_dataset, batch_size=32, shuffle=True)

### Training

In [16]:
num_epochs = 50

In [17]:
input_size= len(train_feat_dataset.variable_columns)
output_size= 2
num_epochs = 50

parameters = {
    'learning_rate': 0.001,
    'dense_units': 128,
    'dense_layers': 1,
    'dropout_rate': 0.2,
    'early_stopping_patience': 7
}

model = FullyConnectedClassifier(input_size= input_size, output_size= output_size, parameters= parameters)

In [18]:
model.train_model(train_loader, test_loader, num_epochs)

Epoch 1/50, Training Loss: 0.6128, Training Accuracy: 0.6752, Validation Loss: 0.5994
Epoch 2/50, Training Loss: 0.5854, Training Accuracy: 0.6965, Validation Loss: 0.5975
Epoch 3/50, Training Loss: 0.5785, Training Accuracy: 0.6970, Validation Loss: 0.5859
Epoch 4/50, Training Loss: 0.5701, Training Accuracy: 0.7013, Validation Loss: 0.5884
Epoch 5/50, Training Loss: 0.5697, Training Accuracy: 0.7028, Validation Loss: 0.5910
Epoch 6/50, Training Loss: 0.5649, Training Accuracy: 0.7043, Validation Loss: 0.5873
Epoch 7/50, Training Loss: 0.5598, Training Accuracy: 0.7098, Validation Loss: 0.5820
Epoch 8/50, Training Loss: 0.5593, Training Accuracy: 0.7081, Validation Loss: 0.5899
Epoch 9/50, Training Loss: 0.5536, Training Accuracy: 0.7093, Validation Loss: 0.5834
Epoch 10/50, Training Loss: 0.5505, Training Accuracy: 0.7133, Validation Loss: 0.5844
Epoch 11/50, Training Loss: 0.5521, Training Accuracy: 0.7095, Validation Loss: 0.5829
Epoch 12/50, Training Loss: 0.5526, Training Accurac

### Evaluation

In [19]:
test_predictions = model.predict(test_loader)

In [20]:
actual_labels = []
for _, labels in test_loader:
    actual_labels.extend(labels.tolist())

In [21]:
correct_predictions = sum(p == t for p, t in zip(test_predictions, actual_labels))
accuracy = correct_predictions / len(actual_labels)
print(f"Accuracy: {accuracy:.4f}")

Accuracy: 0.5000


In [22]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Assuming your task is a classification task
precision = precision_score(actual_labels, test_predictions, average='macro')
recall = recall_score(actual_labels, test_predictions, average='macro')
f1 = f1_score(actual_labels, test_predictions, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(actual_labels, test_predictions)
print("Confusion Matrix:\n", conf_matrix)

Precision: 0.5000
Recall: 0.5000
F1-Score: 0.4998
Confusion Matrix:
 [[435 403]
 [436 404]]


In [ ]:
klk

## RQ2

In [ ]:
### Data preparation

# Reduce the Dataset for the datapoints concerning rq2

rq2_train_feat_dataset= train_feat_dataset.copy()
processed_train_indexes, train_targets= ped.get_indexes_and_targets_by_rq(2, rq2_train_feat_dataset.features)
rq2_train_feat_dataset.features= rq2_train_feat_dataset.features.iloc[processed_train_indexes]
rq2_train_feat_dataset.features.reset_index(drop=True, inplace=True)

rq2_test_feat_dataset= test_feat_dataset.copy()
processed_test_indexes, test_targets= ped.get_indexes_and_targets_by_rq(2, rq2_test_feat_dataset.features)
rq2_test_feat_dataset.features= rq2_test_feat_dataset.features.iloc[processed_test_indexes]
rq2_test_feat_dataset.features.reset_index(drop=True, inplace=True)

# Reduce the features that are correlated in the training data
train_cols= rq2_train_feat_dataset.reduce_features(targets= train_targets, corr_threshold=0.75)
rq2_test_feat_dataset.keep_only_specified_variable_columns(train_cols)

# Normalize features
normalization_params= rq2_train_feat_dataset.normalize_features()
rq2_test_feat_dataset.apply_normalization(normalization_params)

rq2_train_feat_dataset.objective_features.head()

rq2_test_feat_dataset.objective_features.head()

train_loader= rq2_train_feat_dataset.get_variable_features_loader(train_targets)
test_loader= rq2_test_feat_dataset.get_variable_features_loader(test_targets)

# Train dataset
print_counts_and_percentages(train_targets)

# Test dataset
print_counts_and_percentages(test_targets)

### Training

num_epochs = 50

input_size= len(rq2_train_feat_dataset.variable_columns)
output_size= 3
num_epochs = 50

parameters = {
    'learning_rate': 0.0003,
    'dense_units': 128,
    'dense_layers': 2,
    'dropout_rate': 0.2
}

model = FullyConnectedClassifier(input_size= input_size, output_size= output_size, parameters= parameters)

model.train_model(train_loader, test_loader, num_epochs)

### Evaluation

test_predictions = model.predict(test_loader)

actual_labels = []
for _, labels in test_loader:
    actual_labels.extend(labels.tolist())

correct_predictions = sum(p == t for p, t in zip(test_predictions, actual_labels))
accuracy = correct_predictions / len(actual_labels)
print(f"Accuracy: {accuracy:.4f}")


from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Assuming your task is a classification task
precision = precision_score(actual_labels, test_predictions, average='macro')
recall = recall_score(actual_labels, test_predictions, average='macro')
f1 = f1_score(actual_labels, test_predictions, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(actual_labels, test_predictions)
print("Confusion Matrix:\n", conf_matrix)

## RQ3

In [ ]:
### Data preparation

# Reduce the Dataset for the datapoints concerning rq3

rq3_train_feat_dataset= train_feat_dataset.copy()
processed_train_indexes, train_targets= ped.get_indexes_and_targets_by_rq(3, rq3_train_feat_dataset.features)
rq3_train_feat_dataset.features= rq3_train_feat_dataset.features.iloc[processed_train_indexes]
rq3_train_feat_dataset.features.reset_index(drop=True, inplace=True)

rq3_test_feat_dataset= test_feat_dataset.copy()
processed_test_indexes, test_targets= ped.get_indexes_and_targets_by_rq(3, rq3_test_feat_dataset.features)
rq3_test_feat_dataset.features= rq3_test_feat_dataset.features.iloc[processed_test_indexes]
rq3_test_feat_dataset.features.reset_index(drop=True, inplace=True)

# Reduce the features that are correlated in the training data
train_cols= rq3_train_feat_dataset.reduce_features(targets= train_targets, corr_threshold=0.75)
rq3_test_feat_dataset.keep_only_specified_variable_columns(train_cols)

# Normalize features
normalization_params= rq3_train_feat_dataset.normalize_features()
rq3_test_feat_dataset.apply_normalization(normalization_params)

rq3_train_feat_dataset.objective_features.head()

rq3_test_feat_dataset.objective_features.head()

train_loader= rq3_train_feat_dataset.get_variable_features_loader(train_targets)
test_loader= rq3_test_feat_dataset.get_variable_features_loader(test_targets)

# Train dataset
print_counts_and_percentages(train_targets)

# Test dataset
print_counts_and_percentages(test_targets)

### Training

num_epochs = 50

input_size= len(rq3_train_feat_dataset.variable_columns)
output_size= 4
num_epochs = 50

parameters = {
    'learning_rate': 0.0003,
    'dense_units': 64,
    'dense_layers': 1,
    'dropout_rate': 0
}

model = FullyConnectedClassifier(input_size= input_size, output_size= output_size, parameters= parameters)

model.train_model(train_loader, test_loader, num_epochs)

### Evaluation

test_predictions = model.predict(test_loader)

actual_labels = []
for _, labels in test_loader:
    actual_labels.extend(labels.tolist())

correct_predictions = sum(p == t for p, t in zip(test_predictions, actual_labels))
accuracy = correct_predictions / len(actual_labels)
print(f"Accuracy: {accuracy:.4f}")


from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Assuming your task is a classification task
precision = precision_score(actual_labels, test_predictions, average='macro')
recall = recall_score(actual_labels, test_predictions, average='macro')
f1 = f1_score(actual_labels, test_predictions, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(actual_labels, test_predictions)
print("Confusion Matrix:\n", conf_matrix)

## RQ4

In [ ]:
### Data preparation

# Reduce the Dataset for the datapoints concerning rq4

rq4_train_feat_dataset= train_feat_dataset.copy()
processed_train_indexes, train_targets= ped.get_indexes_and_targets_by_rq(4, rq4_train_feat_dataset.features)
rq4_train_feat_dataset.features= rq4_train_feat_dataset.features.iloc[processed_train_indexes]
rq4_train_feat_dataset.features.reset_index(drop=True, inplace=True)

rq4_test_feat_dataset= test_feat_dataset.copy()
processed_test_indexes, test_targets= ped.get_indexes_and_targets_by_rq(4, rq4_test_feat_dataset.features)
rq4_test_feat_dataset.features= rq4_test_feat_dataset.features.iloc[processed_test_indexes]
rq4_test_feat_dataset.features.reset_index(drop=True, inplace=True)

# Reduce the features that are correlated in the training data
train_cols= rq4_train_feat_dataset.reduce_features(targets= train_targets, corr_threshold=0.75)
rq4_test_feat_dataset.keep_only_specified_variable_columns(train_cols)

# Normalize features
normalization_params= rq4_train_feat_dataset.normalize_features()
rq4_test_feat_dataset.apply_normalization(normalization_params)

rq4_train_feat_dataset.objective_features.head()

rq4_test_feat_dataset.objective_features.head()

train_loader= rq4_train_feat_dataset.get_variable_features_loader(train_targets)
test_loader= rq4_test_feat_dataset.get_variable_features_loader(test_targets)

# Train dataset
print_counts_and_percentages(train_targets)

# Test dataset
print_counts_and_percentages(test_targets)

### Training

num_epochs = 50

input_size= len(rq4_train_feat_dataset.variable_columns)
output_size= 4
num_epochs = 50

parameters = {
    'learning_rate': 0.0003,
    'dense_units': 64,
    'dense_layers': 2,
    'dropout_rate': 0.2
}

model = FullyConnectedClassifier(input_size= input_size, output_size= output_size, parameters= parameters)

model.train_model(train_loader, test_loader, num_epochs)

### Evaluation

test_predictions = model.predict(test_loader)

actual_labels = []
for _, labels in test_loader:
    actual_labels.extend(labels.tolist())

correct_predictions = sum(p == t for p, t in zip(test_predictions, actual_labels))
accuracy = correct_predictions / len(actual_labels)
print(f"Accuracy: {accuracy:.4f}")


from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# Assuming your task is a classification task
precision = precision_score(actual_labels, test_predictions, average='macro')
recall = recall_score(actual_labels, test_predictions, average='macro')
f1 = f1_score(actual_labels, test_predictions, average='macro')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(actual_labels, test_predictions)
print("Confusion Matrix:\n", conf_matrix)